# IMPORT STATEMENTS

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import os
import matplotlib.pyplot as plt
import graphviz
from operator import itemgetter
from networkx.algorithms import approximation as approx
from networkx.algorithms import bipartite

# DATA LOADING

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Labels
combined_phenotype= '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Metadata/phenotypic_data.csv'
labels_mappings= pd.read_csv(combined_phenotype)
labels_mappings

In [ ]:
ub = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Preprocessed Data/UCLA Binary 0_2'
uw = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Preprocessed Data/UCLA Weighted v2'

In [ ]:
# Add the directories as per requirement
dirs = [ub]

graphs = list()
labels = list()

for dir in dirs:
    for file in os.listdir(dir):
        if file.endswith(".csv"):
            if (dir == ub):
                subject = file[12:16]
            elif (dir == uw):
                subject = file[14:18]
            else:
                print("If you are using a statically binarized folder:")
                print("Add a condition along with cb condition for COBRE and ub condition for UCLA")
                print("Example: elif (dir == ub) becomes elif (dir == ub) or (dir == ub0_3)")
                exit(0)

            # APPENDING LABEL
            mask = labels_mappings['Subject'] == int(subject)
            labels.append((labels_mappings[mask]['Label'].values[0], int(subject)))

            # APPENDING CORRESPONDING GRAPH - may change based on implementation
            df = pd.read_csv(dir+file, header=None)
            G = nx.from_pandas_adjacency(df)
            graphs.append(G)          

# INDIVIDUAL SUBJECT EXPLORATION

## Basic information about the graph

In [ ]:
G = nx.from_pandas_adjacency(df)

In [ ]:
print("Number of nodes in the graph:", len(G.nodes()))
print("Nodes:", G.nodes())
print()
print("Number of edges in the graph:", len(G.edges()))
print("Edges:", G.edges(data=True))

In [ ]:
plt.figure(3,figsize=(20,12)) 
nx.draw(G, with_labels=True, node_size=700, font_size=8)
plt.show()

## Global measures

### Chain decomposition

In [ ]:
ch = nx.chain_decomposition(G)
ch = list(ch)
ch

In [ ]:
reachable = {}
for l in ch:
  for t in l:
    s=t[0]
    d=t[1]
    if s not in reachable:
      reachable[s]=[d]
    else:
      reachable[s].append(d)

    if d not in reachable:
      reachable[d]=[s]
    else:
      reachable[d].append(s)

max_l = 0
highest = -1
for i in sorted (reachable) :
    print ((i, reachable[i]), end =" ")
    if len(reachable[i])>max_l:
      max_l = len(reachable[i])
      highest = i
    print()

print("Most connected node: ", highest, " connected to ", max_l," nodes")

# Utility function (for measures' calculations)

In [ ]:
class global_measures_binary:

  def __init__(self, subject_no, schiz, G, df):
    self.G = G
    self.subject = subject_no
    if schiz:
      self.schiz = 'Schizophrenic'
    else:
      self.schiz = 'Control'
    self.df = df
    self.df.at[self.subject, "Subject"] = self.subject
    self.df.at[self.subject, "Schizophrenic"] = int(schiz)
    
    print('\n\nSUBJECT NUMBER: ',self.subject)
   
    self.approx_node_connectivity()
    self.approx_max_independent_sets()
    self.approx_max_clique()
    self.approx_clique_removal()
    self.approx_large_clique_size()
    self.approx_avg_clustering()
    self.approx_diameter()
    self.approx_min_edge_dominating_set()
    self.approx_min_wt_dominatind_set()
    self.approx_maximal_matching()
    self.approx_ramsay()
    self.approx_tsp()
    self.approx_treewidth_min_degree()
    self.approx_treewidth_min_fill_in()
    self.approx_min_wt_vertex_cover()
    self.approx_randomised_partitioning()
    self.approx_one_exchange()
    self.assortativity()
    self.avg_shortest_path_length()
    self.asteroidal_triplet()
    self.bridges()
    self.local_bridges()
    self.colouring()
    self.avg_degree()
    self.global_clustering()
    self.edges()
    self.chordal()
    self.cliques()
    self.num_of_isolates()
    self.non_randomness()
    self.transitivity()
    self.connected_components()
    self.diameter()
    self.k_edge_connected(1)
    self.k_edge_connected(2)
    self.k_edge_connected(3)

  def approx_node_connectivity(self):
    print("Approximation and Heuristics Node Connectivity")
    r = approx.node_connectivity(self.G)
    self.df.at[self.subject, "Approximation and Heuristics Node Connectivity"] = r

  def approx_k_components(self):
    print("Approximation and Heuristics K Components")
    r = approx.k_components(self.G)
    self.df.at[self.subject, "Approximation and Heuristics K Components"] = r

  def approx_max_independent_sets(self):
    print("Approximation and Heuristics Max Independent Sets")
    r = approx.maximum_independent_set(self.G)
    self.df.at[self.subject, "Approximation and Heuristics Max Independent Sets"] = str(r)

  def approx_max_clique(self):
    print("Approximation and Heuristics Max Cliques")
    r = approx.max_clique(self.G)
    self.df.at[self.subject, "Approximation and Heuristics Max Cliques"] = str(r)

  def approx_clique_removal(self):
    print("Approximation and Heuristics Clique Removal")
    r = approx.clique_removal(self.G)
    self.df.at[self.subject, "Approximation and Heuristics Clique Removal"] = str(r)

  def approx_large_clique_size(self):
    print("Approximation and Heuristics Large Clique Size")
    r = approx.large_clique_size(self.G)
    self.df.at[self.subject, "Approximation and Heuristics Large Clique Size"] = r

  def approx_avg_clustering(self):
    print("Approximation and Heuristics Average Clustering")
    r = approx.average_clustering(self.G, trials=1000, seed=None)
    self.df.at[self.subject, "Approximation and Heuristics Average Clustering"] = r

  def approx_diameter(self):
    print("Approximation and Heuristics Diameter")
    r = approx.diameter(self.G, seed=None)
    self.df.at[self.subject, "Approximation and Heuristics Diameter"] = r

  def approx_min_edge_dominating_set(self):
    print("Approximation and Heuristics Min Edge Dominating Set")
    r = approx.min_edge_dominating_set(self.G)
    self.df.at[self.subject, "Approximation and Heuristics Min Edge Dominating Set"] = str(r)

  def approx_min_wt_dominatind_set(self):
    print("Approximation and Heuristics Min Wt Dominating Set")
    r = approx.min_weighted_dominating_set(self.G, weight=None)
    self.df.at[self.subject, "Approximation and Heuristics Min Wt Dominating Set"] = str(r)

  def approx_maximal_matching(self):
    print("Approximation and Heuristics Maximal Matching")
    r = nx.maximal_matching(self.G)
    self.df.at[self.subject, "Approximation and Heuristics Maximal Matching"] = str(r)

  def approx_ramsay(self):
    print("Approximation and Heuristics Ramsay")
    r = approx.ramsey_R2(self.G)
    self.df.at[self.subject, "Approximation and Heuristics Ramsay"] = str(r)

  def approx_tsp(self):
    print("Approximation and Heuristics TSP")
    r = approx.traveling_salesman_problem(self.G, weight='weight', nodes=None, cycle=True, method=None)
    self.df.at[self.subject, "Approximation and Heuristics TSP"] = str(r)
  
  def approx_treewidth_min_degree(self):
    print("Approximation and Heuristics treewidth Min Degree")
    r = approx.treewidth_min_degree(self.G)
    self.df.at[self.subject, "Approximation and Heuristics treewidth Min Degree"] = r[0]

  def approx_treewidth_min_fill_in(self):
    print("Approximation and Heuristics treewidth Min Fill In")
    r = approx.treewidth_min_degree(self.G)
    self.df.at[self.subject, "Approximation and Heuristics treewidth Min Fill In"] = r[0]

  def approx_min_wt_vertex_cover(self):
    print("Approximation and Heuristics treewidth Min Wt Vertex Cover")
    r = approx.min_weighted_vertex_cover(self.G, weight=None)
    self.df.at[self.subject, "Approximation and Heuristics treewidth Min Wt Vertex Cover"] = str(r)

  def approx_randomised_partitioning(self):
    print("Approximation and Heuristics Randomized Partitioning")
    r = approx.randomized_partitioning(self.G, seed=None, p=0.5, weight=None)
    self.df.at[self.subject, "Approximation and Heuristics Randomized Partitioning"] = str(r)

  def approx_one_exchange(self):
    print("Approximation and Heuristics One Exchange")
    r = approx.one_exchange(self.G, initial_cut=None, seed=None, weight=None)
    self.df.at[self.subject, "Approximation and Heuristics One Exchange"] = str(r)

  def assortativity(self):
    print("Degree Assortativity")
    r = nx.degree_assortativity_coefficient(self.G)
    self.df.at[self.subject, "Degree Assortativity"] = r

  def avg_shortest_path_length(self):
    print("Avg Shortest Path Length")
    if nx.number_connected_components(self.G) is 1:
      r = nx.average_shortest_path_length(self.G)
      self.df.at[self.subject, "Avg Shortest Path Length"] = r
    else:
      self.df.at[self.subject, "Avg Shortest Path Length"] = np.NaN

  def asteroidal_triplet(self):
    print("Asteroidal Triple")
    if not nx.is_at_free(self.G):
      r = nx.find_asteroidal_triple(self.G)
    else:
      r = []
    self.df.at[self.subject, "Asteroidal Triple"] = str(r)

  def is_bipartite(self):
    print("Is Bipartite")
    r = nx.is_bipartite(self.G)
    self.df.at[self.subject, "Is Bipartite"] = r

  def bipartite_color(self):
    print("Bipartite Color")
    r = bipartite.color(self.G)
    self.df.at[self.subject, "Bipartite Color"] = str(r)

  def bridges(self):
    print("Bridges")
    r = nx.has_bridges(self.G)
    self.df.at[self.subject, "Bridges"] = r

  def print_bridges(self):
    r = nx.has_bridges(self.G)
    if r:
      print(self.subject,"->",self.schiz,"-> has bridges: ", list(nx.bridges(self.G)))

  def local_bridges(self):
    print("Local bridges")
    r = len(set(nx.local_bridges(self.G)))
    self.df.at[self.subject, "Local Bridges"] = r

  def colouring(self):
    print("Graph coloring")
    r = nx.greedy_color(self.G)
    self.df.at[self.subject, "Graph colouring"] = max(r.values())

  def avg_degree(self):
    print("Avg Degree")
    N,K = self.G.order(), self.G.size()
    self.df.at[self.subject, "Avg Degree"] = float(K)/N

  def global_clustering(self):
    print("Global Clustering")
    r = nx.average_clustering(self.G)
    self.df.at[self.subject, "Global Clustering"] = r

  def edges(self):
    print("Edges")
    r = len(self.G.edges())
    self.df.at[self.subject, "Edges"] = r

  def chordal(self):
    print("Chordal")
    r = nx.is_chordal(self.G)
    self.df.at[self.subject, "Chordal"] = r

  # maximal cliques
  def cliques(self):
    print("Maximal Cliques")
    r = nx.graph_number_of_cliques(self.G)
    self.df.at[self.subject, "Maximal Cliques"] = r
  
  # maximal cliques
  def print_cliques(self):
    r = list(nx.find_cliques(self.G))
    print(self.subject,"->",self.schiz,"-> has cliques: ", r)

  def num_of_isolates(self):
    print("Num of isolates")
    r = nx.number_of_isolates(self.G)
    self.df.at[self.subject, "Num of isolates"] = r

  def non_randomness(self):
    print("Non Randomness wrt Random Model")
    if nx.number_connected_components(self.G) is 1:
      nr, nr_rd = nx.non_randomness(self.G)
      self.df.at[self.subject, "Non Randomness wrt Random Model"] = nr_rd
    else:
      self.df.at[self.subject, "Non Randomness wrt Random Model"] = np.NaN

  def print_chain_decomposition(self):
    r = list(nx.chain_decomposition(self.G))
    print(self.subject,"->",self.schiz,"-> has chain: ", r)

  def transitivity(self):
    print("Transitivity")
    r = nx.transitivity(self.G)
    self.df.at[self.subject, "Transitivity"] = r

  def connected_components(self):
    print("Connected Components")
    r = nx.number_connected_components(self.G)
    self.df.at[self.subject, "Connected Components"] = r

  def k_edge_connected(self, k):
    print(str(k)+" Edge Connected")
    r = nx.is_k_edge_connected(self.G, k=k)
    self.df.at[self.subject, str(k)+" Edge Connected"] = r

  def diameter(self):
    print("Diameter")
    r = nx.diameter(self.G)
    self.df.at[self.subject, "Diameter"] = r

# Driver function

In [ ]:
df = pd.DataFrame()
for i, g in enumerate(graphs):
  global_measures_binary(labels[i][1], labels[i][0], g, df)
df.to_csv('global_measures_binary.csv')

Streaming output truncated to the last 5000 lines.
Num of isolates
Non Randomness wrt Random Model
Transitivity
Connected Components
Diameter
1 Edge Connected
2 Edge Connected
3 Edge Connected


SUBJECT NUMBER:  2048
Approximation and Heuristics Node Connectivity
Approximation and Heuristics Max Independent Sets
Approximation and Heuristics Max Cliques
Approximation and Heuristics Clique Removal
Approximation and Heuristics Large Clique Size
Approximation and Heuristics Average Clustering
Approximation and Heuristics Diameter
Approximation and Heuristics Min Edge Dominating Set
Approximation and Heuristics Min Wt Dominating Set
Approximation and Heuristics Maximal Matching
Approximation and Heuristics Ramsay
Approximation and Heuristics TSP
Approximation and Heuristics treewidth Min Degree
Approximation and Heuristics treewidth Min Fill In
Approximation and Heuristics treewidth Min Wt Vertex Cover
Approximation and Heuristics Randomized Partitioning
Approximation and Heuristics One Exc

# Results

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df

,Subject,Schizophrenic,Approximation and Heuristics Node Connectivity,Approximation and Heuristics Max Independent Sets,Approximation and Heuristics Max Cliques,Approximation and Heuristics Clique Removal,Approximation and Heuristics Large Clique Size,Approximation and Heuristics Average Clustering,Approximation and Heuristics Diameter,Approximation and Heuristics Min Edge Dominating Set,Approximation and Heuristics Min Wt Dominating Set,Approximation and Heuristics Maximal Matching,Approximation and Heuristics Ramsay,Approximation and Heuristics TSP,Approximation and Heuristics treewidth Min Degree,Approximation and Heuristics treewidth Min Fill In,Approximation and Heuristics treewidth Min Wt Vertex Cover,Approximation and Heuristics Randomized Partitioning,Approximation and Heuristics One Exchange,Degree Assortativity,Avg Shortest Path Length,Asteroidal Triple,Bridges,Local Bridges,Graph colouring,Avg Degree,Global Clustering,Edges,Chordal,Maximal Cliques,Num of isolates,Non Randomness wrt Random Model,Transitivity,Connected Components,Diameter,1 Edge Connected,2 Edge Connected,3 Edge Connected
2019,2019.0,0.0,43.0,"{128, 0, 4, 147, 89, 156}","{11, 13, 14, 15, 16, 17, 21, 22, 23, 26, 27, 2...","({128, 0, 4, 147, 89, 156}, [{4, 133, 6, 5, 16...",39.0,0.693,2.0,"{(54, 55), (100, 101), (40, 41), (72, 73), (18...","{2, 136, 14, 16, 20, 21, 22, 26, 27, 28, 29, 3...","{(54, 55), (100, 101), (40, 41), (72, 73), (18...","({4, 133, 6, 5, 16, 21, 44, 45, 50, 51, 57, 59...","[0, 160, 0, 86, 81, 1, 159, 1, 99, 69, 82, 85,...",127.0,127.0,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(3769, ({0, 1, 2, 3, 4, 5, 7, 9, 12, 14, 17, 1...","(4076, ({0, 1, 2, 6, 11, 14, 15, 16, 17, 20, 2...",0.320875,1.437977,"[104, 115, 128]",False,0.0,49.0,45.804878,0.688333,7512.0,False,227045.0,0.0,7.277249,0.698336,1.0,2.0,True,True,True
2024,2024.0,0.0,62.0,"{33, 4, 134, 8, 12, 56}","{5, 6, 7, 10, 145, 44, 45, 46, 47, 48, 51, 52,...","({33, 4, 134, 8, 12, 56}, [{0, 1, 2, 131, 3, 6...",31.0,0.644,2.0,"{(54, 55), (86, 87), (100, 101), (154, 156), (...","{96, 98, 101, 102, 133, 139, 144, 117, 91, 62,...","{(54, 55), (86, 87), (100, 101), (154, 156), (...","({0, 1, 2, 131, 3, 6, 18, 25, 29, 32, 49, 53, ...","[0, 161, 4, 130, 41, 7, 92, 73, 1, 108, 56, 15...",144.0,144.0,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(3699, ({2, 3, 6, 7, 9, 14, 20, 21, 22, 23, 24...","(4000, ({1, 2, 3, 5, 6, 11, 12, 13, 14, 15, 16...",0.173836,1.448825,"[80, 125, 134]",False,0.0,43.0,44.920732,0.643601,7367.0,False,165809.0,0.0,2.495979,0.644565,1.0,2.0,True,True,True
2022,2022.0,0.0,42.0,"{1, 100, 5, 36, 159}","{0, 1, 2, 131, 4, 133, 3, 135, 140, 24, 30, 32...","({1, 100, 5, 36, 159}, [{0, 1, 2, 131, 4, 133,...",32.0,0.659,2.0,"{(54, 55), (86, 87), (154, 156), (40, 41), (72...","{3, 131, 135, 136, 144, 24, 32, 33, 38, 45, 46...","{(54, 55), (86, 87), (154, 156), (40, 41), (72...","({0, 1, 2, 131, 4, 133, 3, 135, 140, 24, 30, 3...","[0, 161, 162, 1, 79, 4, 82, 81, 150, 12, 6, 10...",140.0,140.0,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(3750, ({8, 10, 12, 13, 14, 16, 19, 22, 25, 26...","(4069, ({0, 3, 4, 8, 10, 13, 14, 15, 17, 18, 1...",0.158501,1.438875,"[39, 70, 131]",False,0.0,42.0,45.731707,0.668338,7500.0,False,261112.0,0.0,3.893600,0.666596,1.0,2.0,True,True,True
2015,2015.0,0.0,60.0,"{128, 0, 76, 148, 29}","{6, 7, 8, 9, 10, 145, 146, 147, 21, 150, 157, ...","({128, 0, 76, 148, 29}, [{128, 129, 134, 12, 1...",30.0,0.649,2.0,"{(54, 55), (86, 87), (100, 101), (125, 126), (...","{32, 162, 163, 100, 145, 149, 150, 151, 152, 5...","{(54, 55), (86, 87), (100, 101), (125, 126), (...","({128, 129, 134, 12, 13, 14, 15, 16, 17, 22, 1...","[0, 163, 0, 158, 1, 107, 61, 1, 108, 54, 1, 12...",131.0,131.0,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(3400, ({0, 1, 2, 3, 4, 6, 10, 12, 14, 16, 17,...","(3712, ({0, 1, 4, 5, 6, 11, 12, 13, 14, 15, 19...",0.091587,1.497755,"[29, 44, 0]",False,0.0,44.0,40.932927,0.658575,6713.0,False,33730.0,0.0,0.766125,0.65